# Where are we in the layer?

* Hardware
* Memory Manager
* **Process Manager**
* Process coordination (semaphores)
* IPC
* ...

# Processes and Non-determinism
* Events happen that are hard to predict
    * Processes perform syscalls -> traps
    * Processes generate faults -> trap or exception
    * HW interrupts
        * user interupts (eg. mouse + keyboard)
        * service request (eg. harddisk)
* Events are hard to track and service times vary greatly

## How to manage non-determinsm

Try to make things seem more deterministic so that we can control the chaos

### Making things deterministic
* Limit number of entry points to the kernel
    * Think of access control to a building. If you only have one set of doors into a building, it's easier to control flow. 
    * entry point is an address where code begins executing from
* Use queues to store events
    * This is just like how people manage events. eg. ticket queues, bank queues
    * Many events can be processed in an orderly manner
* **The goal of the dispatcher is to manage things at the entry store** (eg. traffic light, store greeter)

### How to map this to an OS

Use a process model:
* Process is a person, it wants a service
* Services can be a syscall (trap), an interrupt, or a fault
* Assume when executing in the OS (eg. Kernel mode), interrupts are ignored until back in user mode
* Assume there is always a null/idle process ready to run
    * We cannot just stay in the kernel, otherwise we will never catch interrupts
* Interrupts:
    * Since interrupts are now forced to take place in user mode, this implies some process is running when an interrupt occurs
    * Therefore, that interrupt, from the perspective of the kernel, can be considered to be coming from that process just like a trap (ie. software interrupt)
    * Everything now looks the same. Interrupts, faults, traps all come from userland from a process. We can now all take the same generic first steps in the dispatcher

# Structure: Context Switcher and Dispatcher

Two main components
* Context switcher (CS) - mechanism
* Dispatcher - policy coordinator
    * Dispathcer selects 1st process to run and calls CS
    * CS eventually does a return to the dispatcher

## What happens when a syscall/interrupt occurs
1. CS gets control, saves information about running process
2. CS reloads kernel context
3. CS returns to dispatcher
4. Dispatcher determines the nature of the service request
5. Invokes the appropriate service
6. The service can:
    * complete the request
    * block the process by putting it on a queue
7. Service routine returns to dispatcher
8. Dispatcher selects next process to run
9. CS switches to process

## What is a process?
Use a process control block to contain all info the kernel needs about a process to manage it

1. PID
2. State (what the process is doing/wants)
3. Priority
4. Parent (depends on system. Not needed in our kernel)
5. CPU state (registers/flags) or state's location (a pointer to the state)
6. Owned resources (eg. open files, semaphores, message spaces, physical memmaps)
    * When the process dies, we want the kernel to reclaim all of the process's resources

We don't need to put all of this stuff in for Assignment 1 (eg. priority). Also, more info will be added as we increase kernel functionality.

## PCB Management
* PCBs are maintained in a global process table
    * array of PCBs declared within the kernel code (historically constant)
    * having an upper limit to the number of processes that can exist is a good idea to protect yourself from `forkbombs`. Once this limit is reached, do not spawn new processes. Goddammit this would've been handy last year.
* Pass around pointers to PCBs
    * Pointers to PCBs or indices into the process table are passed between functions that operate on processes or service process requests
    * Queues of processes are represented by queues of pointers to PCBs

## Queue Types
* **Ready queues**: Processes waiting ro run (use CPU)
* **Blocked queues**: Processes blocked waiting for an event
* Not all blocked process are on a blocked queue, maybe you'll just make it unscheduleable
* However, all ready processes must be on a ready queue
* Process removed from blocked queues is placed on the ready queue
* Process removed from ready queue is the next process to be run